# Generate Inteferece Graph

# Make Prediction from DL4RegAlloc

In [6]:
from numpy.random import seed
seed(1)
import tensorflow as tf
from tensorflow.keras import layers
tf.random.set_seed(2)

# Build Model

In [ ]:
seqsize = 100
seq_inputs = layers.Input(shape=(seqsize,100,), dtype='float32')
encoder = layers.Bidirectional(layers.LSTM(512, return_sequences=True, name="lstm_1"))(seq_inputs)
encoder = layers.Dropout(0.2)(encoder)
encoder = layers.Bidirectional(layers.LSTM(256, return_sequences=True, name="lstm_2"))(encoder)
encoder = layers.Dropout(0.2)(encoder)
decoder = layers.Bidirectional(layers.LSTM(128, return_sequences=True, name="lstm_3"))(encoder)
decoder = layers.Dropout(0.2)(decoder)
decoderoutputs = layers.TimeDistributed(layers.Dense(101,activation="softmax"))(decoder)
model = tf.keras.Model(inputs=seq_inputs, outputs=decoderoutputs)

# RegAlloc in LLVM